In [1]:
import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

Using TensorFlow backend.


# Dane

In [2]:
data = pd.read_csv('dane_3-8-12.csv')

data.head(14)

,timestamp,MpKrakAlKras-PM10-1g,Przewidywania1,Przewidywania2,Przewidywania3,Weather,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar),Przewidywania8,Przewidywania12
0,01.01.2017 01:00,213.733,213.868,221.923,244.690,Ice fog.,0,-6,1,0.93,10.24,187.320,121.805
1,01.01.2017 02:00,213.868,221.923,244.690,224.673,Ice fog.,190,-7,2,0.93,10.24,181.194,119.096
2,01.01.2017 03:00,221.923,244.690,224.673,212.159,Ice fog.,200,-4,3,0.80,10.23,161.198,121.681
3,01.01.2017 04:00,244.690,224.673,212.159,198.375,Ice fog.,210,-7,3,0.86,10.22,136.218,138.353
4,01.01.2017 05:00,224.673,212.159,198.375,191.881,Ice fog.,200,-6,3,0.80,10.22,121.805,150.415
5,01.01.2017 06:00,212.159,198.375,191.881,187.320,Ice fog.,190,-7,2,0.86,10.22,119.096,161.587
6,01.01.2017 07:00,198.375,191.881,187.320,181.194,Ice fog.,210,-8,3,0.86,10.21,121.681,197.133
7,01.01.2017 08:00,191.881,187.320,181.194,161.198,Ice fog.,0,-8,1,0.93,10.21,138.353,217.923
8,01.01.2017 09:00,187.320,181.194,161.198,136.218,Ice fog.,200,-6,2,0.86,10.21,150.415,250.248
9,01.01.2017 10:00,181.194,161.198,136.218,121.805,Ice fog.,200,-3,2,0.74,10.21,161.587,227.516


In [3]:
data.Weather.unique()
weather_dict = {}
weathers = data.Weather.unique()

for num, name in enumerate(weathers, start=1):
    weather_dict[name] = num
#weather_dict


In [29]:
data["hour"] = data["timestamp"].apply(lambda x:x[-8:-6]) #pora dnia chyba ma znaczenie (godziny szczytu itd)
data["month"] = data["timestamp"].apply(lambda x:x[5:7]) #pora roku też może mieć wpływ

X = data[["hour","month","MpKrakAlKras-PM10-1g","Direction(deg)","Temp(C)","Wind(m/s)","Humidity","Barometer(mbar)"]]

y = data[["Przewidywania3","Przewidywania8","Przewidywania12"]]

In [36]:
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X_scale

,hour,month,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,-0.990722,-1.0,-0.105539,-1.230197,-1.640738,-1.070174,0.921077,1.009479
1,-0.990722,-1.0,-0.105538,0.460831,-1.746423,-0.627967,0.921077,1.009479
2,-0.990722,-1.0,-0.105499,0.549833,-1.429368,-0.185761,0.163555,0.881552
3,-0.990722,-1.0,-0.105388,0.638834,-1.746423,-0.185761,0.513181,0.753625
4,-0.990722,-1.0,-0.105486,0.549833,-1.640738,-0.185761,0.163555,0.753625
...,...,...,...,...,...,...,...,...
17268,1.009365,-1.0,-0.106544,1.172843,-0.795258,0.256445,0.921077,1.009479
17269,1.009365,-1.0,0.108162,1.350846,-0.795258,0.256445,0.571452,1.137406
17270,1.009365,-1.0,-0.106527,1.439848,-0.689574,0.698652,0.221826,1.265333
17271,1.009365,-1.0,-0.106528,1.439848,-0.689574,1.140858,0.221826,1.265333


In [37]:
#weather_id trzeba by dodać jako one-hot a nie słownik, tylko że wtedy mamy 100 dodatkowych kolumn

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.1,
                                                    random_state=0)

In [38]:
X_scale.head(10)
#y.head(10)

,hour,month,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,-0.990722,-1.0,-0.105539,-1.230197,-1.640738,-1.070174,0.921077,1.009479
1,-0.990722,-1.0,-0.105538,0.460831,-1.746423,-0.627967,0.921077,1.009479
2,-0.990722,-1.0,-0.105499,0.549833,-1.429368,-0.185761,0.163555,0.881552
3,-0.990722,-1.0,-0.105388,0.638834,-1.746423,-0.185761,0.513181,0.753625
4,-0.990722,-1.0,-0.105486,0.549833,-1.640738,-0.185761,0.163555,0.753625
5,-0.990722,-1.0,-0.105547,0.460831,-1.746423,-0.627967,0.513181,0.753625
6,-0.990722,-1.0,-0.105614,0.638834,-1.852107,-0.185761,0.513181,0.625697
7,-0.990722,-1.0,-0.105646,-1.230197,-1.852107,-1.070174,0.921077,0.625697
8,-0.990722,-1.0,-0.105668,0.549833,-1.640738,-0.627967,0.513181,0.625697
9,-0.990722,-1.0,-0.105698,0.549833,-1.323683,-0.627967,-0.186070,0.625697


# Model

In [39]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='linear'))

In [40]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

In [41]:
model.fit(X_train, y_train, epochs=50, batch_size=20, verbose = 1)

Epoch 1/5
15545/15545 [==============================] - 1s 74us/step - loss: 27388296454.4630 - mse: 27388286976.0000 - mae: 14227.2646
Epoch 2/5
15545/15545 [==============================] - 1s 57us/step - loss: 27386585916.3187 - mse: 27386576896.0000 - mae: 14220.0645
Epoch 3/5
15545/15545 [==============================] - 1s 56us/step - loss: 27377553686.0362 - mse: 27377539072.0000 - mae: 14316.1172
Epoch 4/5
15545/15545 [==============================] - 1s 57us/step - loss: 27358249129.5929 - mse: 27358169088.0000 - mae: 14580.5010
Epoch 5/5
15545/15545 [==============================] - 1s 59us/step - loss: 27329005221.4644 - mse: 27329001472.0000 - mae: 15012.9854


In [45]:
y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms/np.mean(y)

Przewidywania3     3430.044462
Przewidywania8        8.772765
Przewidywania12       8.772775
dtype: float64

In [46]:
now = np.array([-0.990722,-1.0,-0.105646,-1.230197,-1.852107,-1.070174,0.921077,0.625697])
np.transpose(now)
pred = model.predict(now)
pred
#def predict(condtions):
#    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=96)
#    score = model.predict([x_test])[0]

ValueError: Error when checking input: expected dense_4_input to have shape (8,) but got array with shape (1,)